In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyomo.environ import *

In [2]:
# Dados Entrada
g = ['Produtor A', 'Produtor B']
cap_max = [100,100] # MW
ng = len(g)

scen = ['Cenario 1', 'Cenario 2']
prob_cenario = [0.05,0.95] #Aqui modelei probabilidade de cenario pra diferentes cargas, basta adicionar uma nova linha
demanda_reserva = [20,10] # por cenario
ns = len(scen)

consumidor = ['Consumidor 1']
nc = len(consumidor)
demanda_energia = [130]

preco_energia = [10,30] # $/MWh
preco_reserva = [0,25]  # $/MW

corte = ['Corte de Carga 1']  #Aqui também ficou possível adicionar cortes diferentes com preços diferentes
ncorte = len(corte)
preco_corte = [100]      # $/MWh

# Criar Modelo Concreto - Mercado de Energia
model_energia = ConcreteModel()

model_energia.ger = Set(initialize=g)
model_energia.price = Param(model_energia.ger, initialize={g[i]: preco_energia[i] for i in range(ng)})
model_energia.pmax = Param(model_energia.ger, initialize={g[i]: cap_max[i] for i in range(ng)})

model_energia.carga = Set(initialize=consumidor)
model_energia.dem = Param(model_energia.carga, initialize={consumidor[i]: demanda_energia[i] for i in range(nc)})

#Criar Modelo Concreto - Mercado de Reserva com Cenários Probabilísticos
model_reserva = ConcreteModel()

model_reserva.ger = Set(initialize=g)
model_reserva.price = Param(model_reserva.ger, initialize={g[i]: preco_energia[i] for i in range(ng)})
model_reserva.price_res = Param(model_reserva.ger, initialize={g[i]: preco_reserva[i] for i in range(ng)})
model_reserva.pmax = Param(model_reserva.ger, initialize={g[i]: cap_max[i] for i in range(ng)})

model_reserva.carga = Set(initialize=consumidor)
model_reserva.cenario = Set(initialize=scen)
model_reserva.dem_res = Param(model_reserva.cenario, initialize={scen[j]: demanda_reserva[j] for j in range(ns)})
model_reserva.prob = Param(model_reserva.cenario, initialize={scen[j]: prob_cenario[j] for j in range(ns)})

model_reserva.corte = Set(initialize=corte)
model_reserva.price_cut = Param(model_reserva.corte, initialize={corte[i]: preco_corte[i] for i in range(ncorte)})

# Declarar Variáveis - Mercado de Energia
model_energia.P = Var(model_energia.ger, domain=NonNegativeReals)

# Declarar Variáveis - Mercado de Reserva
model_reserva.R = Var(model_reserva.ger, domain=NonNegativeReals)
model_reserva.r = Var(model_reserva.ger, model_reserva.cenario, domain=NonNegativeReals)
model_reserva.L = Var(model_reserva.corte, model_reserva.cenario, domain = NonNegativeReals)

# Funções Objetivo

#FOB do Mercado de Energia
def FOB_energia(model):
    #Parte Determinística
    FOB = sum(model.price[g] * model.P[g]  for g in model.ger)
    return FOB
model_energia.objective = Objective(rule=FOB_energia(model_energia), sense=minimize)

#FOB do Mercado de Reserva
def FOB_reserva(model):
    #Parte Determinística
    FOB_det = sum(model.price_res[g] * model.R[g]  for g in model.ger)
    #Parte Probabilística
    FOB_prob = 0
    for i in model.cenario:
        FOB_prob += model.prob[i] * (sum(model.price[g] * model.r[g,i] for g in model.ger)
                                    +sum(model.price_cut[j] * model.L[j,i] for j in model.corte))
    return FOB_det + FOB_prob

model_reserva.objective = Objective(rule=FOB_reserva(model_reserva), sense=minimize)

#Restrições do Mercado de Energia
model_energia.balanco = Constraint(expr=sum(model_energia.P[g] for g in model_energia.ger) == sum(model_energia.dem[j] for j in model_energia.carga))

model_energia.limites = ConstraintList()
for g in model_energia.ger:
    model_energia.limites.add(expr=model_energia.P[g] <= model_energia.pmax[g])

solver = SolverFactory('glpk')
# solver = SolverFactory('ipopt',executable='/content/ipopt')

mercado_energia = solver.solve(model_energia, tee=False)


In [3]:
#Print das Variáveis
print(f'FOB do Mercado de Energia:' ,value(model_energia.objective))
print('Status Final do Problema de Otimização:', mercado_energia.solver.status, '\n')
for i in model_energia.ger:
    print(f'Produção do {i}:' ,value(model_energia.P[i]))

#Restrições do Mercado de Reserva
model_reserva.restr = ConstraintList()
for g in model_reserva.ger:
    model_reserva.restr.add(expr=model_reserva.R[g] + value(model_energia.P[g]) <= model_reserva.pmax[g])  ##Reparar que aqui essa restrição do mercado de reserva depende de uma variável do mercado de energia: Resolver nessa ordem!!
    for i in model_reserva.cenario:
        model_reserva.restr.add(expr=model_reserva.r[g,i] <= model_reserva.R[g])
for i in model_reserva.cenario:
    model_reserva.restr.add(expr=sum(model_reserva.r[g,i] for g in model_reserva.ger) + sum(model_reserva.L[j,i] for j in model_reserva.corte) == model_reserva.dem_res[i])
    model_reserva.restr.add(expr=sum(model_reserva.L[j,i] for j in model_reserva.corte) <= model_reserva.dem_res[i])
mercado_reserva = solver.solve(model_reserva, tee=False)

print(f'\n FOB do Mercado de Reserva:' ,value(model_reserva.objective))

for j in model_reserva.ger:
    print(f'Reserva do {j}:' ,value(model_reserva.R[j]))
    print(f'Custo da Reserva do {j}:' ,value(model_reserva.R[j])*model_reserva.price_res[j])


for i in model_reserva.cenario:
    for j in model_reserva.ger:
        print(f'\nReserva ativada do {j} no {i}:' ,value(model_reserva.r[j,i]))
        print(f'Custo da Reserva Ativada do {j} no {i}:' ,value(model_reserva.r[j,i])*model_reserva.price[j])
    for j in model_reserva.corte:
        print(f'\nCorte de Carga do {i}:' ,value(model_reserva.L[j,i]))
        print(f'Custo do Corte de Carga do {i}:' ,value(model_reserva.L[j,i])*model_reserva.price_cut[j])

FOB do Mercado de Energia: 1900.0
Status Final do Problema de Otimização: ok 

Produção do Produtor A: 100.0
Produção do Produtor B: 30.0

 FOB do Mercado de Reserva: 600.0
Reserva do Produtor A: 0.0
Custo da Reserva do Produtor A: 0.0
Reserva do Produtor B: 10.0
Custo da Reserva do Produtor B: 250.0

Reserva ativada do Produtor A no Cenario 1: 0.0
Custo da Reserva Ativada do Produtor A no Cenario 1: 0.0

Reserva ativada do Produtor B no Cenario 1: 10.0
Custo da Reserva Ativada do Produtor B no Cenario 1: 300.0

Corte de Carga do Cenario 1: 10.0
Custo do Corte de Carga do Cenario 1: 1000.0

Reserva ativada do Produtor A no Cenario 2: 0.0
Custo da Reserva Ativada do Produtor A no Cenario 2: 0.0

Reserva ativada do Produtor B no Cenario 2: 10.0
Custo da Reserva Ativada do Produtor B no Cenario 2: 300.0

Corte de Carga do Cenario 2: 0.0
Custo do Corte de Carga do Cenario 2: 0.0
